In [6]:
%%file app.py

import pickle
import numpy as np
from flask import Flask, request, jsonify

class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = [random.uniform(-1.0, 1.0) for _ in range(1+X.shape[1])] 
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update = self.eta*(target-self.predict(xi))
                self.w_[1:] += update*xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0, 1, -1)

# Create a flask
app = Flask(__name__)


# Create an API end point
@app.route('/predict', methods=['GET', 'POST'])
def get_prediction():
                                
    if request.method == 'GET':                            
        sepal_length = float(request.args.get('sl'))
        petal_length = float(request.args.get('pl'))
    elif request.method == 'POST':
        data = request.get_json(force=True)
        sepal_length = float(data.get('sl'))
        petal_length = float(data.get('pl'))
        
    
    features = [sepal_length, petal_length]

    # Load pickled model file
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
        
    # Predict the class using the model
    predicted_class = int(model.predict(features))
    
    # Return a json object containing the features and prediction
    output = dict(features=features, predicted_class=predicted_class)
    return jsonify(output)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

Overwriting app.py


In [7]:
import requests
response = requests.get("http://127.0.0.1:8000/predict?sl=6.3&pl=2.6")

print(response.content)

b'{"features":[6.3,2.6],"predicted_class":-1}\n'
